In [1]:
import pandas as pd
import numpy as np
import zipfile
from gensim.models import Word2Vec
from random import sample
import pickle

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import GRU
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
zf = zipfile.ZipFile('/content/drive/My Drive/Data Hack/Data/train_mddNHeX.zip')
train = pd.read_csv(zf.open('train.csv'))

zf = zipfile.ZipFile('/content/drive/My Drive/Data Hack/Data/test_HLxMpl7.zip')
test = pd.read_csv(zf.open('test.csv'))

In [3]:
train=train.groupby('user_id').challenge.apply(lambda x: ' '.join(x)).reset_index()
train.head()

,user_id,challenge
0,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...
1,4580,CI23663 CI23855 CI23933 CI23975 CI24530 CI2371...
2,4581,CI26155 CI26156 CI26157 CI26158 CI26159 CI2616...
3,4582,CI23855 CI24915 CI24917 CI23933 CI23663 CI2495...
4,4585,CI23855 CI23975 CI24917 CI25135 CI23848 CI2371...


In [4]:
test=test.groupby('user_id').challenge.apply(lambda x: ' '.join(x)).reset_index()
test.head()

,user_id,challenge
0,4577,CI23855 CI23933 CI24917 CI24915 CI23714 CI2366...
1,4578,CI23663 CI23855 CI24917 CI23933 CI23975 CI2371...
2,4579,CI26939 CI26940 CI26941 CI26942 CI26943 CI2694...
3,4583,CI23663 CI23855 CI23975 CI23714 CI23848 CI2393...
4,4584,CI23855 CI23975 CI25135 CI23848 CI23714 CI2491...


In [5]:
sentence_list=pd.concat((train.challenge,test.challenge)).apply(lambda x: x.split())

In [ ]:
embed=Word2Vec(sentence_list, size=500, window=5, workers=8,min_count=0, hs=1,iter=100,sg=1)
embed.save('/content/drive/My Drive/Data Hack/Embedding')

In [7]:
embed= Word2Vec.load('/content/drive/My Drive/Data Hack/Embedding')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
pretrained_weights = embed.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

Result embedding shape: (5502, 500)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [9]:
def word2idx(word):
    return embed.wv.vocab[word].index
def idx2word(idx):
    return embed.wv.index2word[idx]

In [10]:
train_seq=np.array(train.challenge.apply(lambda x: x.split()).apply(lambda x: ([(word2idx(y)) for y in x])).tolist())
test_seq=np.array(test.challenge.apply(lambda x: x.split()).apply(lambda x: ([(word2idx(y)) for y in x])).tolist())

In [11]:
x_train=train_seq[:,:10]
y_train= train_seq[:,10]
x_train.shape, y_train.shape

((69532, 10), (69532,))

In [12]:
x_train=np.concatenate((x_train,x_train,x_train),axis=0)
y_train=np.concatenate((train_seq[:,10],train_seq[:,11],train_seq[:,12]))

In [13]:
embed_size=500
vocab_size=5502
keras_model = Sequential()
keras_model.add(Embedding(input_dim=vocab_size, output_dim=embed_size, 
                    weights=[pretrained_weights]))
keras_model.add(GRU(units=embed_size))
keras_model.add(Dense(units=vocab_size))

keras_model.add(Activation('softmax'))
keras_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [14]:
sample_weight=np.concatenate((np.ones(train.shape[0])*3,np.ones(train.shape[0])*2,np.ones(train.shape[0])))

In [ ]:
keras_model.fit(x_train,y_train, epochs=3, sample_weight=sample_weight,
          batch_size=512, verbose=1)

In [17]:
pickle.dump(keras_model.to_json(),open('gru.pkl','wb'))

In [18]:
keras_model.save_weights('gru.weights')

In [22]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return preds.argsort()[-3:][::-1]
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    retar= probas[0].argsort()[-3:][::-1]
    return retar

def generate_next(text):
    word_idxs = [word2idx(word) for word in text.split()]
    prediction = keras_model.predict(x=np.array(word_idxs))
    idxl = sample(prediction[-1], temperature=0.0)
    return [idx2word(idx) for idx in idxl]

In [24]:
test.challenge.values[0]

generate_next('CI23855 CI23933 CI24917 CI24915 CI23714 CI23663 CI24958 CI25135 CI25727 CI24530')

test_inds=np.array(test.challenge.apply(lambda x: np.array([word2idx(word) for word in x.split()])).tolist())


nnpred=keras_model.predict(test_inds)
nnpred2= pd.Series(nnpred.argsort(axis=1)[:,-3:].tolist())

count=0
test['pred']=nnpred2.apply(lambda x: [idx2word(y) for y in x[::-1]])
test_sub= test.copy()
test_sub=pd.concat((test_sub,test_sub,test_sub))
test_sub=test_sub.sort_values('user_id').reset_index(drop=True)
test_sub['seq']=test_sub.groupby('user_id').cumcount()
test_sub.seq= test_sub.seq+11
test_sub['user_sequence']= test_sub.user_id.astype('str') + '_' + test_sub.seq.astype('str')
test_sub['challenge']=test_sub.apply(lambda row: row['pred'][row['seq']-11], axis=1)
test_sub[['user_sequence','challenge']].to_csv('submission.csv',index=False)